# Breaking Silos

UN Datathon 2023

Sources:

- [CO2 Emissionen Gebäude](https://www.bafu.admin.ch/bafu/de/home/themen/klima/fachinformationen/verminderungsmassnahmen/gebaeude/co2-rechner-gebaeude.html#accordion1699022038096) (download .CSV format, unzip into the `data` folder)


In [1]:
import os
import warnings 
import pandas as pd
from pyproj import Proj, transform
from matplotlib import pyplot as plt

In [2]:
F_BUILDINGS = os.path.join('data', 'MAPGEO_GEB_CO2.txt')

# Read in the basic dataset
df_buildings = pd.read_csv(F_BUILDINGS, header=0, sep='\t')
'Loaded: %d rows' % (len(df_buildings))

'Loaded: 1790170 rows'

In [3]:
F_ADDRESSES = os.path.join('data', 'MAPGEO_DOM_CO2.txt')

# Read in the basic dataset
df_addresses = pd.read_csv(F_ADDRESSES, header=0, sep='\t')
'Loaded: %d rows' % (len(df_addresses))

'Loaded: 1861915 rows'

In [4]:
df_addresses.head(1)

,EGID,EDID,STRSP,DEXPDAT,STRNAME_DEINR,DPLZ4_DPLZNAME
0,564,0,DE,01.11.2023,Alte Hedingerstrasse 39,8910 Affoltern am Albis


In [5]:
# Clean up some columns we're not interested in
if 'LINKCO2' in df_buildings:
    df_buildings = df_buildings.drop('LINKCO2', axis=1)
if 'LINKBAFU' in df_buildings:
    df_buildings = df_buildings.drop('LINKBAFU', axis=1)
if 'LINKPDF' in df_buildings:
    df_buildings = df_buildings.drop('LINKPDF', axis=1)
if 'HINWEIS' in df_buildings:
    df_buildings = df_buildings.drop('HINWEIS', axis=1)
df_buildings.head(5)

,EGID,GKODE,GKODN,GSTAT,GKAT,GWAERZH1,GENH1,GWAERSCEH1,CO2_CLASS,CO2_RANGE,GUE20,GEXPDAT
0,11513432,2679649.268,1237500.347,1004,1020,7430.0,7530.0,860.0,7,> 25,1,01.11.2023
1,11513433,2680637.900,1236936.229,1004,1020,7430.0,7530.0,860.0,7,> 25,1,01.11.2023
2,11517090,2679018.828,1235828.156,1004,1020,7431.0,7530.0,860.0,7,> 25,1,01.11.2023
3,1600000,2679679.869,1237453.256,1004,1020,7430.0,7530.0,869.0,7,> 25,1,01.11.2023
4,1600001,2679694.997,1237491.284,1004,1030,7436.0,7530.0,869.0,7,> 25,0,01.11.2023


In [6]:
len(df_buildings)

1790170

In [7]:
bbox = df_buildings[(df_buildings.GKODE > 2512875.0) | (df_buildings.GKODN > 1131325.0)].index
df_geneva = df_buildings.drop(bbox)
len(df_geneva)

47708

In [8]:
# Convert from CH1903+ (EPSG:4150) to WGS84 (EPSG:4326)
warnings.filterwarnings('ignore') # yees, we know.. there be dragons ...
pWorld = Proj(init='epsg:4326')
pCH = Proj(init='epsg:2056')

(lon, lat) = transform(pCH, pWorld, 2679684.826, 1237581.429)
(lat, lon) # quick test ..

(47.284316436317695, 8.491945346821495)

In [19]:
ENERGY_LEVELS = df_geneva['CO2_RANGE'].unique().tolist()

df_geneva['lat'] = 0
df_geneva['lon'] = 0
df_geneva['co2'] = 0

for ix, bld in df_geneva.iterrows():
    (lon, lat) = transform(pCH, pWorld, bld['GKODE'], bld['GKODN'])
    df_geneva['lat'][ix] = lat
    df_geneva['lon'][ix] = lon
    df_geneva['co2'][ix] = ENERGY_LEVELS.index(bld['CO2_RANGE'])

df_geneva.head(2)

,EGID,GKODE,GKODN,GSTAT,GKAT,GWAERZH1,GENH1,GWAERSCEH1,CO2_CLASS,CO2_RANGE,GUE20,GEXPDAT,lat,lon,co2
1515802,11527367,2499308.0,1130106.0,1004,1020,7430.0,7530.0,860.0,7,> 25,1,01.11.2023,46.31479,6.131373,0
1515803,280002108,2499582.0,1130077.0,1004,1030,NaN,7520.0,NaN,7,> 25,1,01.11.2023,46.31457,6.134935,0


In [20]:
F_GENEVA = os.path.join('data', 'MAPGEO_GEB_CO2_GENEVA.txt')
df_geneva.to_csv(F_GENEVA)